In [13]:
using LinearAlgebra, Distributions, Random

In [2]:
one_vec = ones(2);
zero_vec = zeros(2);

### MAP creation

In [3]:
function check_is_MAP(C,D)
    all(D .>= 0.0) || return false
    all(diag(C) .< 0.0) || return false
    all(C-Diagonal(C) .>= 0.0) ||return false
    iszero(sum(C+D,dims=2)) || return false   #will be redundent in the make_C_D function and in the similarity...
    return true
end;

$$
\boldsymbol{\alpha} = 
\frac{1}{(\lambda_{21}+ \lambda_{22}) q_{12}+ (\lambda_{11}+ \lambda_{12}) q_{21}}
\Big[q_{21} \lambda_{11} + q_{12} \lambda_{21}, ~~
q_{21} \lambda_{12} + q_{12} \lambda_{22}
 \Big].
 $$


In [4]:
function make_C_D_α(q12, q21, λ11, λ12, λ21, λ22)
    C = [
        -q12 - λ11      q12 - λ12;
         q21 - λ21   -q21 - λ22
    ]
    D = [
        λ11  λ12;
        λ21  λ22
    ]
    check_is_MAP(C,D) || error("problem with parameters")

    α = ([q21*λ11 + q12*λ21, q21*λ12 + q12*λ22]') / ((λ21 + λ22)*q12 + (λ11 + λ12)*q21)
    return C, D, α
end

q12 = 10
q21 = 20
λ11 = 1.0
λ12 = 2.0
λ21 = 3.0
λ22 = 4.0

C, D, α = make_C_D_α(q12, q21, λ11, λ12, λ21, λ22);

In [5]:
C

2×2 Matrix{Float64}:
 -11.0    8.0
  17.0  -24.0

In [6]:
D

2×2 Matrix{Float64}:
 1.0  2.0
 3.0  4.0

In [7]:
Q = C+D

2×2 Matrix{Float64}:
 -10.0   10.0
  20.0  -20.0

In [8]:
Q*one_vec

2-element Vector{Float64}:
 0.0
 0.0

In [9]:
P = inv(-C)*D

2×2 Matrix{Float64}:
 0.375     0.625
 0.390625  0.609375

In [10]:
sum(P, dims = 2)

2×1 Matrix{Float64}:
 0.9999999999999998
 1.0

In [11]:
α*P - α #indeed stationary vector

1×2 adjoint(::Vector{Float64}) with eltype Float64:
 -5.55112e-17  0.0

### Similarity matrix

$$
S {\mathbf 1} = {\mathbf 1}
$$
so also,

$$
{\mathbf 1} = S^{-1}{\mathbf 1}
$$


In [12]:
function make_rand_S_matrix()
    s1, s2 = rand(Uniform(-1,1), 2)
    S = [s1 1-s1
         1-s2 s2]
    return S, inv(S)
end;

In [18]:
Random.seed!(1)
S, Sinv = make_rand_S_matrix();

In [19]:
S

2×2 Matrix{Float64}:
 -0.853267   1.85327
  1.30152   -0.301517

In [20]:
Sinv

2×2 Matrix{Float64}:
 0.139929  0.860071
 0.604013  0.395987

In [21]:
S*one_vec

2-element Vector{Float64}:
 1.0
 1.0

In [22]:
Sinv*one_vec

2-element Vector{Float64}:
 0.9999999999999999
 1.0

### Transform MAP

$$
C_{\text{new}} = S^{-1} C S
$$


$$
D_{\text{new}} = S^{-1} D S
$$

$$
Q_{\text{new}} = C_{\text{new}} + D_{\text{new}} = S^{-1} (C + D) S
$$

So $Q_{\text{new}} {\mathbf 1} = 0$.

$$
\pi Q = 0
$$

Claim:

$$
\pi S Q_{\text{new}} = 0.
$$

Hence $\pi S$ is the $\pi_{\text{new}}$.

$$
\lambda^* = \pi D {\mathbf 1}
$$

$$
\lambda^*_{\text{new}} = \pi_{\text{new}} D_{\text{new}} {\mathbf 1} = \pi S S^{-1} D S {\mathbf 1} = \lambda^*
$$


$$
\alpha = \frac{1}{\lambda^*} \pi D
$$

$$
\alpha_{\text{new}} = \frac{1}{\lambda^*_{\text{new}}} \pi_{\text{new}} D_{\text{new}}
= \frac{1}{\lambda^*} \pi S S^{-1} D S = \alpha S
$$

In [23]:
function find_similar_map(C,D; max_attempts = 10^3)
    attempts = 0
    while attempts <= max_attempts
        S, Sinv = make_rand_S_matrix()
        Cnew = Sinv*C*S
        Dnew = Sinv*D*S
        is_map = check_is_MAP(Cnew, Dnew)
        if is_map
            @info "Found similar map after $attempts attempts"
            return S, Sinv, Cnew, Dnew
        end
        attempts += 1
    end
    error("did not find a similar one")
end;

In [25]:
Random.seed!(1)
S, Sinv, Cnew, Dnew = find_similar_map(C,D);

[ Info: Found similar map after 7 attempts


In [26]:
check_is_MAP(Cnew, Dnew)

true

In [27]:
Qnew = Cnew+Dnew

2×2 Matrix{Float64}:
 -19.6378   19.6378
  10.3622  -10.3622

In [28]:
sum(Qnew, dims = 2)

2×1 Matrix{Float64}:
 0.0
 0.0

In [29]:
Pnew = inv(-Cnew)*Dnew

2×2 Matrix{Float64}:
 0.516815  0.483185
 0.53244   0.46756

In [30]:
sum(Pnew, dims = 2)

2×1 Matrix{Float64}:
 1.0000000000000004
 1.0000000000000004

In [31]:
αnew = α*S

1×2 adjoint(::Vector{Float64}) with eltype Float64:
 0.524248  0.475752

In [32]:
αnew*Pnew-αnew

1×2 adjoint(::Vector{Float64}) with eltype Float64:
 2.22045e-16  1.66533e-16

### Actual probablity law (check on first two intervals)

In [33]:
# single interval (T_0)
pdf_original_1(t) = α*exp(C*t)*D*one_vec
pdf_new_1(t) = αnew*exp(Cnew*t)*Dnew*one_vec;

In [34]:
maximum(abs, [pdf_original_1(t) - pdf_new_1(t) for t in 0:0.01:2.5])

3.9968028886505635e-15

In [35]:
# two intervals (T_0 and T_1)
pdf_original_2(t0, t1) = α*exp(C*t0)*D*exp(C*t1)*D*one_vec
pdf_new_2(t0, t1) = αnew*exp(Cnew*t0)*Dnew*exp(Cnew*t1)*Dnew*one_vec;

In [36]:
maximum(abs, [pdf_original_2(t0, t1) - pdf_new_2(t0, t1) for t0 in 0:0.01:2.5 for t1 in 0:0.01:2.5])

2.1316282072803006e-14

### LST

In [39]:
function find_LST_params(α, C, D) #currently just for p=2
    a0 = det(C)
    a1 = -tr(C)
    b1 = α*D*one_vec
    c11 = α*D*D*one_vec
    return (a0, a1, b1, c11)
end;

In [40]:
find_LST_params(α, C, D)

(128.0, 35.0, 5.461538461538462, 29.307692307692307)

In [41]:
find_LST_params(αnew, Cnew, Dnew)

(127.99999999999996, 35.0, 5.461538461538463, 29.307692307692317)

In [62]:
lst_params = []
example_Ss = []
for _ in 1:3
    S, Sinv, Cnew, Dnew = find_similar_map(C,D)
    S = round.(S, digits = 7)
    push!(example_Ss, S)
    αnew = α*S
    new_params = find_LST_params(αnew, Cnew, Dnew)
    push!(lst_params, round.(new_params, digits = 7))
    pdf_original_2(t0, t1) = α*exp(C*t0)*D*exp(C*t1)*D*one_vec
    pdf_new_2(t0, t1) = αnew*exp(Cnew*t0)*Dnew*exp(Cnew*t1)*Dnew*one_vec
    err = maximum(abs, [pdf_original_2(t0, t1) - pdf_new_2(t0, t1) for t0 in 0:0.01:2.5 for t1 in 0:0.01:2.5])
    @show err
end
lst_params

[ Info: Found similar map after 1 attempts


err = 9.944373502435155e-7
err = 4.414847616374118e-7

[ Info: Found similar map after 12 attempts



err = 9.942505130311474e-8

[ Info: Found similar map after 7 attempts


3-element Vector{Any}:
 (128.0, 35.0, 5.4615387, 29.3076933)
 (128.0, 35.0, 5.4615384, 29.3076919)
 (128.0, 35.0, 5.4615384, 29.3076922)

In [61]:
example_Ss

3-element Vector{Any}:
 [0.070525 0.929475; 0.7364981 0.2635019]
 [-0.2185026 1.2185026; 1.6593606 -0.6593606]
 [-0.0374937 1.0374937; 0.9927069 0.0072931]

In [67]:
inv(example_Ss[3])*one_vec

2-element Vector{Float64}:
 1.0
 1.0